### 決定木　パラメータチューニング

In [1]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import  mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree.export import export_text
from sklearn import preprocessing

import graphviz

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.tree.export module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
pd.__version__

'1.0.3'

In [4]:
train_pkl.shape

(55323, 15)

In [5]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [6]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

### 検証するパラメータの設定

In [7]:
grid_param = {
    'criterion': ['mse'],
    'splitter': ['best', 'random'],
    'max_depth': [i for i in range(1, 11, 2)],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [ ]:
grid_param = {
    'criterion': ['mse'],
    'splitter': ['best', 'random'],
    'max_depth': [i for i in range(5, 6)],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [ ]:
def rmse(y, y_pred):
    ret = np.sqrt(mean_squared_error(y, y_pred))
    print("RMSE=", ret)
    return ret

### グリッドサーチ

In [8]:
gs = GridSearchCV(estimator=DecisionTreeRegressor(**grid_param), param_grid=grid_param, scoring=make_scorer(mean_squared_error, greater_is_better=False), cv=5, return_train_score=False)

In [9]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion=['mse'],
                                             max_depth=[1, 3, 5, 7, 9],
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=[1, 3, 5, 7, 9],
                                             min_samples_split=[2, 4, 6, 8, 10],
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=[42],
                                             splitter=['best', 'random']),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['mse'], 'max_dept

### 一番良かったパラメータで学習

In [10]:
gs.best_score_

-16423.524563328312

In [11]:
np.sqrt(abs(gs.best_score_))

128.15429982379956

In [12]:
gs.best_params_

{'criterion': 'mse',
 'max_depth': 9,
 'min_samples_leaf': 9,
 'min_samples_split': 2,
 'random_state': 42,
 'splitter': 'random'}

In [13]:
# gs.cv_results_['mean_test_score']

In [14]:
DT = DecisionTreeRegressor(**gs.best_params_)

In [15]:
DT = DT.fit(train_X, train_y)

### ツリーの可視化

In [ ]:
# tree.plot_tree(DT.fit(train_X, train_y))

In [ ]:
# export_text(DT, feature_names=train.iloc[:, 1:].columns)

In [ ]:
# train.Survived.name

In [ ]:
# len(train.iloc[:, 1:].columns), train.iloc[:, 1:].columns

In [ ]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(DT, out_file=None,
#                                feature_names=train.iloc[:, 1:-1].columns,
#                                class_names=train.y.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

In [ ]:
# graph = graphviz.Source(dot_data)
# graph

In [ ]:
# graph.write('.\DT.png')

In [ ]:
type(train_X)

In [16]:
DT.feature_importances_

array([0.44670298, 0.26065521, 0.05605432, 0.00591051, 0.04797197,
       0.00060231, 0.00560034, 0.00649808, 0.00053471, 0.        ,
       0.12445423, 0.04387601, 0.00113934])

In [17]:
# 特徴量の重要度が高い順に表示
print("特徴量の重要度が高い順：")
# sorted：reverse=True 降順
print(sorted(
    zip(map(lambda x: round(x, 3), DT.feature_importances_), train.iloc[:, [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]].columns),
    reverse=True))

特徴量の重要度が高い順：
[(0.447, 'bathrooms'), (0.261, 'bedrooms'), (0.124, 'room_type_Shared room'), (0.056, 'beds'), (0.048, 'number_of_reviews'), (0.044, 'property_type_num'), (0.006, 'host_response_rate'), (0.006, 'cancellation_policy_super_strict_30'), (0.006, 'cancellation_policy_strict'), (0.001, 'cancellation_policy_super_strict_60'), (0.001, 'cancellation_policy_moderate'), (0.0, 'room_type_Private room')]


In [18]:
sorted(
    zip(map(lambda x: round(x, 3), DT.feature_importances_), train.iloc[:, [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]].columns),
    reverse=True)

[(0.447, 'bathrooms'),
 (0.261, 'bedrooms'),
 (0.124, 'room_type_Shared room'),
 (0.056, 'beds'),
 (0.048, 'number_of_reviews'),
 (0.044, 'property_type_num'),
 (0.006, 'host_response_rate'),
 (0.006, 'cancellation_policy_super_strict_30'),
 (0.006, 'cancellation_policy_strict'),
 (0.001, 'cancellation_policy_super_strict_60'),
 (0.001, 'cancellation_policy_moderate'),
 (0.0, 'room_type_Private room')]

### テストデータで予測

In [19]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [20]:
test_x.shape, test_y.shape

((11065, 13), (11065,))

In [21]:
pred_y = DT.predict(test_x)

In [22]:
np.sqrt(mean_squared_error(test_y, pred_y))

128.9468810355913

### 検証データで予測

In [23]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [24]:
valid.shape

(18528, 14)

In [25]:
# ID の保存
valid_pass = valid.id.values

In [26]:
valid_X = valid.iloc[:, 1:].values

In [27]:
valid_X.shape, train_X.shape

((18528, 13), (44258, 13))

In [28]:
pred_valid_y = DT.predict(valid_X)

In [29]:
pred_valid_y.shape

(18528,)

In [30]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [31]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [32]:
result_df.to_csv("./tree_9.csv", header=False)